# 03 - DP

In [1]:
import ast
from copy import deepcopy
from functools import reduce

import findspark
from common.const import DATASET, FILEPATH, STAGING_FILENAME
from common.dataframe_utils import array_to_string, string_columns_encoder
from common.utils import (change_case, describe_dataframe_details,
                          detect_continuous_variables, plot_column_value_count,
                          rename_columns)
from common.z_score import abs_z_score, z_score
from pyspark.sql import DataFrame, SparkSession, Window
from pyspark.sql.functions import (array_contains, col, explode, greatest,
                                   monotonically_increasing_id, ntile, size,
                                   udf, when)
from pyspark.sql.types import ArrayType, StringType

In [2]:
findspark.init("/home/ubuntu/spark-3.2.1-bin-hadoop2.7")
spark = SparkSession.builder.appName("basics").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/23 22:59:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/23 22:59:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/05/23 22:59:25 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/05/23 22:59:25 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

## 3.4 Data Integration

In [4]:
filepath_list = [
    FILEPATH.CARBON_EMISSION_AMOUNT,
    FILEPATH.CARBON_EMISSION_HEALTH,
    FILEPATH.CARBON_EMISSION_LIFESTYLE,
    FILEPATH.CARBON_EMISSION_TRAVEL,
    FILEPATH.CARBON_EMISSION_WASTE,
]
df_list = [
    spark.read.csv(filepath, header=True, inferSchema=True)
    for filepath in filepath_list
]
for df in df_list:
    print(
        f"Type: {type(df)} | Shape: ({df.count()}, {len(df.columns)}) | Columns: {df.columns}"
    )
df_id = reduce(lambda x, y: x.join(y, on="ID", how="outer"), df_list)
df_id.show()

Type: <class 'pyspark.sql.dataframe.DataFrame'> | Shape: (10200, 2) | Columns: ['ID', 'CarbonEmission']
Type: <class 'pyspark.sql.dataframe.DataFrame'> | Shape: (10200, 3) | Columns: ['ID', 'Body Type', 'Sex']
Type: <class 'pyspark.sql.dataframe.DataFrame'> | Shape: (10200, 12) | Columns: ['ID', 'Diet', 'How Often Shower', 'Heating Energy Source', 'Social Activity', 'Monthly Grocery Bill', 'How Long TV PC Daily Hour', 'How Many New Clothes Monthly', 'How Long Internet Daily Hour', 'Energy efficiency', 'Recycling', 'Cooking_With']
Type: <class 'pyspark.sql.dataframe.DataFrame'> | Shape: (10200, 5) | Columns: ['ID', 'Transport', 'Vehicle Type', 'Frequency of Traveling by Air', 'Vehicle Monthly Distance Km']
Type: <class 'pyspark.sql.dataframe.DataFrame'> | Shape: (10200, 3) | Columns: ['ID', 'Waste Bag Size', 'Waste Bag Weekly Count']


+---+--------------+-----------+------+-----------+----------------+---------------------+---------------+--------------------+-------------------------+----------------------------+----------------------------+-----------------+--------------------+--------------------+------------+------------+-----------------------------+---------------------------+--------------+----------------------+
| ID|CarbonEmission|  Body Type|   Sex|       Diet|How Often Shower|Heating Energy Source|Social Activity|Monthly Grocery Bill|How Long TV PC Daily Hour|How Many New Clothes Monthly|How Long Internet Daily Hour|Energy efficiency|           Recycling|        Cooking_With|   Transport|Vehicle Type|Frequency of Traveling by Air|Vehicle Monthly Distance Km|Waste Bag Size|Waste Bag Weekly Count|
+---+--------------+-----------+------+-----------+----------------+---------------------+---------------+--------------------+-------------------------+----------------------------+----------------------------+-

In [5]:
df = df_id.drop("ID")
df.show()

+--------------+-----------+------+-----------+----------------+---------------------+---------------+--------------------+-------------------------+----------------------------+----------------------------+-----------------+--------------------+--------------------+------------+------------+-----------------------------+---------------------------+--------------+----------------------+
|CarbonEmission|  Body Type|   Sex|       Diet|How Often Shower|Heating Energy Source|Social Activity|Monthly Grocery Bill|How Long TV PC Daily Hour|How Many New Clothes Monthly|How Long Internet Daily Hour|Energy efficiency|           Recycling|        Cooking_With|   Transport|Vehicle Type|Frequency of Traveling by Air|Vehicle Monthly Distance Km|Waste Bag Size|Waste Bag Weekly Count|
+--------------+-----------+------+-----------+----------------+---------------------+---------------+--------------------+-------------------------+----------------------------+----------------------------+-------------

## 3.1 Data Selection

In [ ]:
df = rename_columns(df=df)

In [ ]:
df = df.drop(
    "energy_efficiency",
    "how_long_t_v_p_c_daily_hour",
    "how_often_shower",
)

In [ ]:
df.show()

## 3.2 Data Cleaning

In [ ]:
df = df.na.fill(
    "not_applicable",
    subset=["vehicle_type"],
).na.drop()

In [ ]:
describe_dataframe_details(df=df, spark=spark)

In [ ]:
def string_to_nested_list(s):
    return ast.literal_eval(s)


# Define the UDF
nested_list_udf = udf(string_to_nested_list, ArrayType((StringType())))
df = df.withColumn("recycling", nested_list_udf(df.recycling))
df = df.withColumn("cooking_with", nested_list_udf(df.cooking_with))
df.show()

In [ ]:
inconsistent_values_pair = []
for column_name in df.columns:
    if df.schema[column_name].dataType == StringType():
        distribution_df = df.groupBy(column_name).count()
        distribution_df = distribution_df.where(
            col("count") / df.count() < 0.005
        ).select(column_name)
        for row in distribution_df.toLocalIterator():
            inconsistent_values_pair.append(
                tuple([next(iter(row.asDict())), next(iter(row.asDict().values()))])
            )
expr = " and ".join(
    f'({value_pair[0]} != "{value_pair[1]}")' for value_pair in inconsistent_values_pair
)
df = df.filter(expr)

In [ ]:
describe_dataframe_details(df=df, spark=spark)

In [ ]:
continuous_columns = detect_continuous_variables(df)
continuous_columns.remove(change_case(DATASET.TARGET))
abs_z_df = abs_z_score(df=df, columns=continuous_columns)
for column_name in abs_z_df.columns:
    abs_z_df = abs_z_df.withColumnRenamed(column_name, column_name + "_abs_z")
abs_z_df = (
    abs_z_df.withColumn(
        "max", greatest(*[abs_z_df[column_name] for column_name in abs_z_df.columns])
    )
    .select("max")
    .withColumn("ID", monotonically_increasing_id())
)
df = (
    df.withColumn("ID", monotonically_increasing_id())
    .join(abs_z_df, on="ID", how="inner")
    .drop("ID")
    .filter(col("max") <= 3)
    .drop("max")
)

In [ ]:
describe_dataframe_details(df=df, spark=spark)

In [ ]:
df.show(truncate=False)

## 3.3 Data Construction

In [ ]:
def explode_list_column(df: DataFrame, column_to_explodes: list[str]) -> DataFrame:
    for column_to_explode in column_to_explodes:
        df_unique_values = (
            df.withColumn("flattened", explode(column_to_explode))
            .select("flattened")
            .distinct()
        )
        value = [row.flattened for row in df_unique_values.collect()]
        for column_name in value:
            df = df.withColumn(
                f"{column_to_explode}_{column_name.lower()}",
                array_contains(df[column_to_explode], column_name),
            )
            df = df.withColumn(
                f"{column_to_explode}_{column_name.lower()}",
                when(
                    df[f"{column_to_explode}_{column_name.lower()}"] == True, 1
                ).otherwise(0),
            )
        df = df.withColumn(f"{column_to_explode}_count", size(df[column_to_explode]))
    return df

In [ ]:
df = explode_list_column(df=df, column_to_explodes=["recycling", "cooking_with"])

In [ ]:
df.show()

In [ ]:
windowSpec = Window.orderBy("monthly_grocery_bill")
df = df.withColumn("monthly_grocery_bill_quantile", ntile(3).over(windowSpec))

In [ ]:
df.show()

In [ ]:
plot_column_value_count(df=df, column_name="monthly_grocery_bill_quantile")

In [ ]:
encoded_df = array_to_string(df=df)

In [ ]:
encoded_df = string_columns_encoder(df=encoded_df)

In [ ]:
encoded_df.show()

In [ ]:
continuous_columns = [
    "monthly_grocery_bill",
    "vehicle_monthly_distance_km",
    "waste_bag_weekly_count",
    "how_many_new_clothes_monthly",
    "how_long_internet_daily_hour",
]
encoded_df_with_z = encoded_df.withColumn("id", monotonically_increasing_id())
z_score_df = z_score(df=encoded_df_with_z, columns=deepcopy(continuous_columns))
encoded_df_with_z = encoded_df_with_z.drop(*continuous_columns)
encoded_df_with_z = encoded_df_with_z.join(z_score_df, on="id", how="inner").drop("id")
encoded_df_with_z.show()

In [ ]:
info_table = describe_dataframe_details(spark=spark, df=encoded_df_with_z)
info_table

## Write Data 

In [ ]:
encoded_df_with_z.repartition(2).write.mode("overwrite").parquet(
    f"{FILEPATH.TEMP_STAGING_PATH}/{STAGING_FILENAME.DP}.parquet"
)